<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Notebooks/Llama2_prod_r1%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Llama2 Production

SUMA 26/09/2023

#Instructions

Create Folder for the experiment on personal Google Drive
1.   Load model from HuggingFace
2.   Gain access to model: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
3.   
4.   
5.   Create folder 'data' in project folder
6.   Save folder CrisiLexT26 in folder 'data'
7.   Data sourced from: https://crisislex.org/data-collections.html#CrisisLexT26
3.   Mount Google Drive
4.   Update code with Paths
5.   Pip install llama-cpp-python package
version 0.1.4.9
6.   Documentation for llama-ccp: https://llama-cpp-python.readthedocs.io/en/latest/api-reference/
6.   Load packages llama_ccp and json
7.   Run Code

In [1]:
# Install Transformer package
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00


In [15]:
# Import Packages
from langchain import PromptTemplate,  LLMChain
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import pandas as pd
import numpy as np
import json
import textwrap
import os
import csv
import time
import re
import matplotlib.pyplot as plt

In [3]:
# Mounting GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.


In [4]:
# Logging onto HuggingFace
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00


In [6]:
# Read (timestamps)

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder
def process_folder(folder_path):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'period' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        subfolder_data = process_folder(subfolder_path)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_p = pd.concat(combined_data, ignore_index=True)


# Remove spaces from column names
combined_df_p.columns = combined_df_p.columns.str.replace(' ', '')

# Rename Columns in prep for left join
combined_df_p.rename(columns={'Tweet-ID': 'Tweet ID'}, inplace=True)

# # Save the combined DataFrame to a CSV file
# output_file = 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)

print(f"Combined timestamps read")


Combined timestamps read


In [7]:
# Read and anontate (tweets)

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder and add folder name as label
def process_folder(folder_path, label):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'labeled' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                df['Label'] = label
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        label = subfolder
        subfolder_data = process_folder(subfolder_path, label)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_l = pd.concat(combined_data, ignore_index=True)

# # Save the combined DataFrame to a CSV file
# output_file = 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)

print(f"Combined tweets read")
# working_df.head(10)


Combined tweets read


In [8]:
# Check
combined_df_p.columns
combined_df_p.head()


,Timestamp,Tweet ID,Included(Y/N)
0,Sun Sep 16 06:07:51 +0000 2012,247215201198436352,Y
1,Sun Sep 16 06:08:01 +0000 2012,247215243137257472,Y
2,Sun Sep 16 06:10:16 +0000 2012,247215809372516353,Y
3,Sun Sep 16 06:39:34 +0000 2012,247223182979911681,Y
4,Sun Sep 16 06:41:20 +0000 2012,247223627584520192,Y


In [9]:
# Extract earliest timestamp

# Convert it to datetime first
combined_df_p['Timestamp'] = pd.to_datetime(combined_df_p['Timestamp'])

# Group by 'Tweet-ID' and aggregate to find the earliest timestamp
earliest_timestamps = combined_df_p.groupby('Tweet ID')['Timestamp'].agg('min').reset_index()

# Display or use the resulting DataFrame 'earliest_timestamps'
# print(earliest_timestamps)

In [10]:
# Left join

# Perform a left join on 'Tweet ID'
result_df = combined_df_l.merge(earliest_timestamps, on='Tweet ID', how='left')

# Display
# print(result_df)
# result_df.shape
result_df.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,Timestamp
0,242883454050648064,"#earthquake M 3.3, Virgin Islands region: Sept...",Not labeled,Not labeled,Not related,2012_Costa_Rica_earthquake,2012-09-04 07:15:02+00:00
1,242887379944366080,@EarthquakeTest update your #earthquake s more,Not labeled,Not labeled,Not related,2012_Costa_Rica_earthquake,2012-09-04 07:30:38+00:00
2,242919634125328384,"RT @RedazioneWebAL: #Terremoto, Costi (Pd): ta...",Not labeled,Not labeled,Not related,2012_Costa_Rica_earthquake,2012-09-04 09:38:48+00:00
3,242920737223106561,"#Earthquake M 2.6, Southern Alaska http://t.co...",Not labeled,Not labeled,Not related,2012_Costa_Rica_earthquake,2012-09-04 09:43:11+00:00
4,242936558158757889,５年６ヶ月長期保存可能なえいようかん5本。http://t.co/ZlSVctfi #eqj...,Not labeled,Not labeled,Not applicable,2012_Costa_Rica_earthquake,2012-09-04 10:46:03+00:00


In [11]:
# Tokenizer
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [17]:
# Prompt Template Formatting
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

Create Test Data Frame

In [20]:
import pandas as pd

data = {
    'TweetID': [25454],
    'TweetText': ['MT @COEmergency Colorado 2-1-1 Recovery and Relief Guides http://t.co/VyeHrWp5Un … #COflood #hmrd ^CB'],
    'InformationSource': ['Government'],
    'InformationType': ['Donations and volunteering'],
    'Informativeness Label': ['Related and informative'],
    'Timestamp': ['2013-09-17 10:43:12+00:00'],
    'answer': ['']
}

df = pd.DataFrame(data)
df

,TweetID,TweetText,InformationSource,InformationType,Informativeness Label,Timestamp,answer
0,25454,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013-09-17 10:43:12+00:00,


The function below is used to generate the answer using Llama and langchain

In [52]:
# Create a function using the code below to generate a answer for each row in the dataframe. Then apply the function to the dataframe to generate the answer and save the output to 'answer' in df.
def generate_answer(text, system_prompt, instruction):
    # # Define the instruction
    # instruction = instruction + row['TweetText']
    #define the prompt template
    prompt_template = get_prompt(instruction, system_prompt)
    #create prompt
    prompt = PromptTemplate(template=prompt_template, input_variables= ["text"])
    #create the language model chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    #generate text using the prompt template
    text = llm_chain.run(text)
    #return the text
    return text

In [54]:
# Apply the function to the dataframe to generate the answer and save the output to 'answer' in df.
system_prompt = "You are an advance assistance that excels at classifying topics. Stop after answering question."
instruction = "Classify the following text for me {text}"
df['answer'] = df.apply(lambda row: generate_answer(row['TweetText'], system_prompt, instruction), axis=1)

In [55]:
df

,TweetID,TweetText,InformationSource,InformationType,Informativeness Label,Timestamp,answer
0,25454,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013-09-17 10:43:12+00:00,"Sure! Based on the provided text, I would cl..."


In [ ]:
def generate_prompt(row):

def generate_prompt(row):
    # Define the instruction
    instruction = f"Classify the following text for me: {row['TweetText']}"

    # Define the system prompt
    system_prompt = "You are an advance assistance that excels at classifying topics. Stop after answering question."

    # Generate the prompt
    prompt_template = get_prompt(instruction, system_prompt)

    # Return the prompt
    return prompt_template

In [ ]:
system_prompt = "You are an advance assistance that excels at classifying topics. Stop after answering question."
instruction = "Classify the following text for me {text}"


In [ ]:
# Grab 10 random samples
filtered_df = result_df.query("Label == '2013_Colorado_floods'")
testing_df = filtered_df.sample(n=10, random_state=42)
testing_df.columns = testing_df.columns.str.replace(' ', '')
testing_df['TweetText'] = testing_df['TweetText'].astype(str)


testing_df.head(12)


In [13]:
# Create an empty 'answer' column in testing_df
testing_df['answer'] = ''

# Loop over the data
for index, row in testing_df.iterrows():
    text_in = row['TweetText']

    # Create the prompt for the current row
    prompt = f"Please return the type of crisis for the following snippet. Answer with only the response in one word and without any explanation.\n```\n{text_in}\n```"

    # Generate the answer using LLMChain
    answer = llm_chain.run(prompt)

    # Update the 'answer' column in the DataFrame
    testing_df.at[index, 'answer'] = answer


NameError: ignored

In [14]:
# Print output
testing_df.head(10)


,TweetID,TweetText,InformationSource,InformationType,Informativeness,Label,Timestamp,answer
25454,379918454104408064,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013_Colorado_floods,2013-09-17 10:43:12+00:00,
25670,380796468728045568,"$CLF Factbox: Floods shut oil, gas operations ...",Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-19 20:52:07+00:00,
25673,380808204420014080,Wow- plowing out the mud from the bike path I ...,Eyewitness,Infrastructure and utilities,Related - but not informative,2013_Colorado_floods,2013-09-19 21:38:45+00:00,
25593,380450807746797568,Proud to be playing this all star event in #Lo...,Outsiders,Donations and volunteering,Related - but not informative,2013_Colorado_floods,2013-09-18 21:58:35+00:00,
25344,379623338672287744,RT @OWHnews: Colorado flooding: Have your frie...,Media,Affected individuals,Related - but not informative,2013_Colorado_floods,2013-09-16 15:10:31+00:00,
25611,380490196464041984,#Jamestown resident says things are disappeari...,Outsiders,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-19 00:35:06+00:00,
25559,380337729331933184,RT @EnviroAction: Colorado #fracking site feat...,NGOs,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-18 14:29:15+00:00,
25446,379836589691506688,Praying for Colorado because its crazy floodin...,Outsiders,Sympathy and support,Related - but not informative,2013_Colorado_floods,2013-09-17 05:17:54+00:00,
25792,381803449840435200,RT @denverpost: Colorado's September floods wi...,Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-22 15:33:30+00:00,
25069,378191675941588992,"RT @sciam: Two Dead, Hundreds Evacuated, Cars ...",Media,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-12 16:21:36+00:00,


In [ ]:
# Test single

text_in = testing_df.loc[10507, 'TweetText']

# Create the prompt for the current row
prompt = f"Please return the type of crisis (one word max) for the following snippet. Answer in the format without any explanation.\n```\n{text_in}\n```"

# Generate the answer using LLMChain
answer = llm_chain.run(prompt)

# print
print(answer)


 Please return the type of crisis (one word max) for the following snippet: [crisis]
Please answer with the following options:
[] - No aspect crisis term exists
"Flood"
"Hurricane"
"Wildfire"
"Earthquake"
"Pandemic"
"Terrorism"
"CyberAttack"
"NaturalDisaster"
"Other"
Please select one option from the above list.


In [ ]:
print(text_in)

MT @COEmergency Colorado 2-1-1 Recovery and Relief Guides http://t.co/VyeHrWp5Un … #COflood #hmrd ^CB


In [ ]:
print("Number of rows in testing_df:", len(testing_df))
print("Index values in testing_df:", testing_df.index.tolist())
print("First few rows of testing_df:")
print(testing_df.head())

Number of rows in testing_df: 10
Index values in testing_df: [10507, 10723, 10726, 10646, 10397, 10664, 10612, 10499, 10845, 10122]
First few rows of testing_df:
                  TweetID                                          TweetText  \
10507  379918454104408064  MT @COEmergency Colorado 2-1-1 Recovery and Re...   
10723  380796468728045568  $CLF Factbox: Floods shut oil, gas operations ...   
10726  380808204420014080  Wow- plowing out the mud from the bike path I ...   
10646  380450807746797568  Proud to be playing this all star event in #Lo...   
10397  379623338672287744  RT @OWHnews: Colorado flooding: Have your frie...   

      InformationSource               InformationType  \
10507        Government    Donations and volunteering   
10723             Media  Infrastructure and utilities   
10726        Eyewitness  Infrastructure and utilities   
10646         Outsiders    Donations and volunteering   
10397             Media          Affected individuals   

              